In [6]:
# 묻지도 따지지도 않고 필요한 압축 도구들을 강제로 설치합니다!
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U transformers

### **Objectives**

1. 실습명 : QLoRA로 모델을 양자화해서 학습하기기
2. 핵심 주제:
    1. 데이터셋 불러오기
    2. base 모델 4 bit 양자화 및 토크나이저 불러오기
    3. Text-to-SQL task 모델 QLoRA fine-tuning
3. 학습 목표 :
    1. Text-to-SQL task 학습을 위한 4 bit base 모델 및 토크나이저를 불러올 수 있다.
    2. base 모델에 QLoRA를 이용하여 4-bit 양자화 fine-tuning을 진행할 수 있다.
4. 학습 개념: 키워드명 :
    1. bitsandbytes
    2. 4-bit
    3. QLoRA fine-tuning
5. 학습 방향 :
  - 양자화라는 개념을 이해하고 이를 통해 모델의 성능 하락을 최소화하면서 모델을 학습합니다.
  - 실습 코드는 조교가 직접 구현한 코드를 참고하여 학습합니다.
  - 해당 실습은 모델을 학습시킬 경우 무엇이 필요하고 어떻게 하면 학습을 효율적으로 할 수 있는지 고민해봅니다.


### **Prerequisites**
```
numpy==2.1.0
pandas==2.2.3
transformers==4.56.0
torch==2.8.0+cu126
accelerate==1.10.1
bitsandbytes==0.49.1
datasets==4.0.0
peft==0.17.1
trl==0.22.2
```

랜덤성을 제어하기 위해 seed를 고정합니다.

In [7]:
import os
import torch
import numpy as np
import random

# tqdm이 텍스트 모드를 사용하도록 설정 (ipywidgets 에러 방지)
os.environ["TQDM_DISABLE"] = "0"
os.environ["TQDM_MININTERVAL"] = "1"

# 시드 설정
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

GPU가 인식이 되고 있는지 확인합니다.

In [8]:
import torch

print(torch.cuda.is_available())  # True/False 반환
# device 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

True
Device: cuda


# 1. 데이터셋 불러오기 및 EDA

- 학습 목표 : 데이터셋을 불러올 수 있다.


첨부한 데이터셋을 불러옵니다. 데이터셋은 이전 강의에서 사용한 데이터셋과 동일합니다.

In [9]:
from datasets import load_dataset

text_to_sql = load_dataset('csv', data_files={
    'train': '/content/train.csv',
    'test': '/content/validation.csv'
})
train_df = text_to_sql["train"].to_pandas()
train_df.head()

,db_id,query,question
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...


저희의 목표는 자연어(text)를 SQL로 변환하는 Text-to-SQL 모델을 학습하는 것입니다. 불필요한 데이터는 제거하고, 필요한 데이터만을 사용하여 학습을 진행합니다.

In [10]:
columns = ["query", "question"]

train_data = train_df[columns]
train_data

,query,question
0,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads..."
2,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac..."
3,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
4,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...
...,...,...
6995,SELECT T1.company_name FROM culture_company AS...,What are all the company names that have a boo...
6996,"SELECT T1.title , T3.book_title FROM movie AS...",Show the movie titles and book titles for all ...
6997,"SELECT T1.title , T3.book_title FROM movie AS...",What are the titles of movies and books corres...
6998,SELECT T2.company_name FROM movie AS T1 JOIN c...,Show all company names with a movie directed i...


# 2. 4-bit base 모델 및 토크나이저 불러오기
- 학습 목표 : 4-bit base 모델과 토크나이저를 불러올 수 있다.
- 학습 개념 : 4-bit base 모델
- 진행하는 실습 요약
    - base 모델 불러오기
    - 토크나이저 불러오기
    - 양자화

사용할 모델은 [HuggingFaceTB/SmolLM2-1.7B-Instruct](https://huggingface.co/HuggingFaceTB/SmolLM2-1.7B-Instruct)입니다.

현재 6GB하의 VRAM에서 학습이 불가능한 모델입니다. 학습시 약 8GB의 VRAM이 필요합니다. 이 경우 QLoRA를 사용하여 학습을 진행할 수 있습니다. QLoRA로 학습을 진행하기 위해서는 모델을 4 bit로 초기화를 해야 합니다.

모델 정밀도(model precisions)와 가장 일반적인 데이터 타입들(float16, float32, bfloat16, int8)에 대해 익숙하지 않다면 , 더 자세한 정보는 [컴퓨터의 소수점 표현](https://www.notion.so/ssunbell/2331806f5bc180f88c5dfe67eaf17738?source=copy_link)를 참고하시길 바랍니다.

FP8(8-bit)과 FP4(4-bit)는 각각 부동 소수점 8비트와 4비트 정밀도를 의미합니다.

먼저 FP8 형식으로 부동 소수점 값을 표현하는 방법을 살펴본 다음, FP4 형식이 어떻게 생겼는지 이해해 보겠습니다. FP8 형식 (FP8 format)은 부동 소수점은 n개의 비트로 구성되며 각 비트는 숫자의 구성 요소(부호, 가수, 지수)를 나타내는 특정 범주에 속합니다. 이들은 다음을 의미합니다.
- FP8(부동 소수점 8) 형식은 "딥러닝을 위한 FP8(FP8 for Deep Learning)"이라는 논문에서 처음 소개되었으며, 두 가지 다른 FP8 인코딩 방식을 가집니다: E4M3 (지수 4비트, 가수 3비트)와 E5M2 (지수 5비트, 가수 2비트).

<img src="./assets/img5.png">

- 비트 수를 32개에서 8개로 줄임으로써 정밀도가 상당히 감소하지만, 두 버전 모두 다양한 상황에서 사용될 수 있습니다.
- E4M3 형식으로 표현할 수 있는 부동 소수점의 잠재적 범위는 -448에서 448 사이입니다.
- E5M2 형식은 지수(exponent)의 비트 수가 늘어나면서 범위가 -57344에서 57344까지 늘어나지만, 표현 가능한 수의 총 개수는 일정하게 유지되므로 정밀도(precision)의 손실이 발생합니다.
- 경험적으로 E4M3는 순전파(forward pass)에 가장 적합하고, 두 번째 버전(E5M2)은 역전파(backward computation)에 가장 적합하다는 것이 입증되었습니다


FP4 정밀도 (FP4 precision in a few words)

- 부호(sign) 비트는 부호(+/-)를 나타내고, 지수(exponent) 비트는 해당 비트로 표현된 정수의 2의 거듭제곱(예: $2^{010} = 2^2 = 4$)을 의미합니다.
- 소수부(fraction) 또는 가수(mantissa)는 '1'로 되어 있는 각 비트에 대해 '활성화'되는 2의 음의 거듭제곱들의 합입니다.
- 만약 비트가 '0'이면 해당 $2^{-i}$ (여기서 i는 비트 시퀀스 내의 위치)의 거듭제곱에 대해서는 소수부가 변경되지 않습니다.
- 예를 들어, 가수 비트가 1010이라면 $(0 + 2^{-1} + 0 + 2^{-3}) = (0.5 + 0.125) = 0.625$가 됩니다.
- 최종 값을 얻기 위해서는 소수부에 1을 더한 뒤 모든 결과를 곱합니다.
- 예를 들어, 지수 비트 2개와 가수 비트 1개가 있을 때 '1101'이라는 표현은 다음과 같습니다:$$-1 * 2^2 * (1 + 2^{-1}) = -1 * 4 * 1.5 = -6$$
- FP4에는 고정된 형식이 없으므로 다양한 가수/지수 조합을 시도해 볼 수 있습니다.
- 일반적으로는 3개의 지수 비트를 사용하는 것이 대부분의 경우 더 좋은 성능을 보입니다. 하지만 때로는 2개의 지수 비트와 1개의 가수 비트를 사용하는 것이 더 나은 성능을 내기도 합니다.

4-bit(FP4) 모델을 불러오기 위해서는 먼저 `BitsAndBytesConfig`에서 4-bit에 대한 설정을 초기화해야 합니다.

`BitsAndBytesConfig`는 `bitsandbytes`라고 하는 라이브러리의 config입니다. 양자화를 지원하는 라이브러리중에서 가장 간단하게 모델 양자화를 지원하고 있으며, 현재는 huggingface에서 `bitsandbytes` 관련된 설정도 지원하고 있으므로 저희는 huggingface에서 `bitsandbytes`를 설정하겠습니다.
1. load_in_4bit=True
- 설명: 모델의 가중치(weights)를 4비트 형식으로 로드하도록 활성화합니다.
- 효과: 일반적으로 모델은 32비트(float32)나 16비트(float16/bfloat16)로 저장되는데, 이를 4비트로 줄여서 로드하므로 GPU 메모리(VRAM) 사용량을 획기적으로 줄여줍니다. (약 1/4 ~ 1/8 수준으로 감소)

2. bnb_4bit_use_double_quant=True
- 설명: 이중 양자화(Double Quantization) 기술을 적용합니다.
- 상세: 양자화를 수행할 때 '양자화 상수(quantization constants)'라는 추가적인 파라미터가 생기는데, 이 상수들마저도 한 번 더 양자화하여 메모리를 절약하는 기법입니다.
- 효과: 파라미터당 평균 약 0.4비트 정도의 추가적인 메모리 절약 효과가 있습니다. 성능 저하는 거의 없으면서 메모리를 극한으로 아끼고 싶을 때 사용합니다.

3. bnb_4bit_quant_type="fp4"
- 설명: 4비트 데이터의 **형식(Data Type)**을 지정합니다.
- 옵션:
    - "fp4": 4-bit Float 형식입니다.
    - "nf4": 4-bit NormalFloat 형식입니다. (참고: QLoRA 논문에서는 사전 학습된 신경망 가중치가 정규분포를 따르는 경향이 있으므로 "nf4"가 정보 손실이 더 적다고 제안합니다.)

4. bnb_4bit_compute_dtype=torch.bfloat16
- 설명: 실제 연산(Computation)을 수행할 때 사용할 데이터 타입을 지정합니다.
- 상세: 가중치는 메모리 절약을 위해 4비트로 '저장'되어 있지만, 행렬 곱셈 같은 실제 연산을 할 때는 다시 4비트에서 실수형(float)으로 복원(Dequantize)해야 합니다. 이때 복원할 목표 타입을 bfloat16으로 설정한 것입니다.
- 효과: float16보다 bfloat16이 표현 가능한 수의 범위(dynamic range)가 넓어 학습 안정성이 높기 때문에, Ampere 아키텍처 이상(예: A100, A10, RTX 30/40 시리즈 등)의 GPU를 사용한다면 bfloat16을 권장합니다.

In [40]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

모델은 [HuggingFaceTB/SmolLM2-1.7B-Instruct](https://huggingface.co/naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-1.5B)를 사용하겠습니다. 이 모델은 스켈레톤 2에서 사용한 모델과 동일한 아키텍쳐이지만, 1.7B 모델 사이즈로 이전 0.36B 모델 사이즈보다 약 5배 더 큰 모델입니다.

`quantization_config`를 추가하여 모델과 토크나이저를 load합니다.

In [12]:
!pip install -U bitsandbytes

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "HuggingFaceTB/SmolLM2-1.7B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
if tokenizer.pad_token is None: # pad_token 설정이 되어있지 않는 경우가 존재합니다.
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map={"":"cuda:0"},
    quantization_config=quantization_config
)

config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/218 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

스켈레톤 2에서는 해당 모델을 불러올 때 메모리를 0.8GB를 차지했다면, 1.7B 모델을 4-bit로 양자화하여 불러온다면 메모리를 약 1.6GB로 차지할 것입니다.

원래 1.7GB 모델을 불러오면 약 4GB를 차지하게 되고 4bit로 불러오면 4배가 줄어들게 되서 1GB가 되어야 할 것 같지만,

모델의 성능 저하를 막기 위해서 Normalization Layers, softmax layer 등 특정 레이어는 4-bit로 만들지 않기 때문에 정확하게 4배가 줄어들지는 않습니다.

In [14]:
!nvidia-smi

Tue Feb 24 02:05:08 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P0             29W /   70W |    3223MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

모델의 max_sequence_length를 확인합니다.

max_position_embeddings의 길이는 position embeddings의 길이를 의미합니다.

position embeddings는 입력 토큰에 순서 정보를 넣는 임베딩이므로 해당 차원이 실제 입력 토큰의 길이와 동일합니다.

In [15]:
max_sequence_length = model.config.max_position_embeddings
max_sequence = [0, 0, 0]
for i, row in train_data.iterrows():
    for i, (k, v) in enumerate(row.items()):
        max_sequence[i] = max(max_sequence[i], len(tokenizer(str(v))["input_ids"]))

print("Max sequence length of model:", max_sequence_length)
print("Max sequence length:", max_sequence)

Max sequence length of model: 8192
Max sequence length: [251, 62, 0]


8192의 길이라면 토크나이징을 한 이후의 `max_length`인 251 + 62 보다 훨씬 크기 때문에 충분히 여유롭습니다.

# 3. Text-to-SQL task 데이터 전처리
- 학습 목표 : chat 형식으로 데이터 전처리를 할 수 있다.
- 학습 개념 : 데이터 전처리
- 진행하는 실습 요약
    - apply_chat_template으로 데이터 전처리하기
    - datasets를 이용하여 Datasets 형식으로 변경하기


허깅페이스 tokenizer에서는 jinja2 포맷의 `chat_template`을 제공합니다.

보통 LLM을 제공하는 회사, 모델마다 `chat_template`이 다릅니다. 또한, 최근에는 `chat_template`을 대부분 지원합니다.

따라서, `chat_template`을 확인하고 `apply_chat_template` 매서드를 이용해서 어떻게 적용이 되는지 확인해봅니다.

In [16]:
if tokenizer.chat_template:
    print("=== chat template 사용 가능 ===\n")
    print(tokenizer.chat_template)
else:
    print("=== chat template 사용 불가능 ===\n")

=== chat template 사용 가능 ===

{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
' }}{% endif %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}


Jinja2 포맷이 이상하긴 하지만 코드를 어느정도 이해하시는 분들이라면 이게 어떤 의미인지를 대충 유추해볼 수 있습니다.

```jinja
{% for message in messages %}
```
어떠한 `iterable` 객체를 `for loop`를 사용하는 것을 봐서는 `list` 등의 객체로 넣어줘야 할거 같습니다.


```jinja
{{'<|im_start|>' + message['role'] + '' + message['content'] + '<|im_end|>' + ''}}
```

`message` 내에 보면 `role`과 `content`라는 키값이 필요한 것을 알 수 있습니다. 여기서 `dictionary` 등의 객체를 사용하면 될거 같습니다.

또한, 앞뒤에 `<|im_start|>`, `<|im_end|>`와 같은 스페셜 토큰이 붙는 것을 확인하실 수 있습니다. 이 부분은 저희가 건들 필요 없이 알아서 생성해줍니다.

`apply_chat_template`의 입력으로 넣기 위해서는 chat(conversations) 형식을 지켜서 입력으로 넣어줘야 합니다.

기존에 불러온 데이터프레임을 chat 형식으로 만들고 `apply_chat_template`을 적용해보겠습니다.

In [17]:
# 학습용 데이터 전처리

example = train_df[columns].iloc[0]
messages = [
    {
        "role": "user",
        "content": example["question"]
    },
    {
        "role": "assistant",
        "content": example["query"]
    }
]

print(messages)
print()
chat = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
print(chat)

[{'role': 'user', 'content': 'How many heads of the departments are older than 56 ?'}, {'role': 'assistant', 'content': 'SELECT count(*) FROM head WHERE age  >  56'}]

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
How many heads of the departments are older than 56 ?<|im_end|>
<|im_start|>assistant
SELECT count(*) FROM head WHERE age  >  56<|im_end|>



보시는 바와 같이 모델의 입력에 맞게 형식을 맞춰서 텍스트를 생성해주는 것을 확인하실 수 있습니다.

주의할 점이 크게 2가지가 있습니다.
1. `assistant`는 정답이기 때문에 학습에서만 입력으로 넣어주고 추론에서는 넣어줘서는 안됩니다.
2. `add_generation_prompt=False`는 `assistant`가 들어갔기 때문에 `False`로 설정합니다. 추론용이라면 `True`로 설정해주셔야 합니다.

In [18]:
## 추론용 데이터 전처리
chat_inference = tokenizer.apply_chat_template(messages[:-1], tokenize = False, add_generation_prompt = True)
print(chat_inference)

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
How many heads of the departments are older than 56 ?<|im_end|>
<|im_start|>assistant



추론용 데이터를 확인해보시면

`SELECT count(*) FROM head WHERE age  >  56<|im_end|>`

이 사라진 것을 확인하실 수 있습니다.

유의할 점은
1. 모델이 정답으로 생성할 텍스트 : `SELECT count(*) FROM head WHERE age  >  56`
2. EOS 스페셜 토큰 : `<|im_end|>`

두가지로 구성되어 있는 `assistant` 부분을 학습에 사용하게 됩니다.


이제 모델 입력으로 넣기 위해서 데이터를 전처리하도록 하겠습니다.

데이터프레임에 있는 데이터를 꺼내서 `Dataset` 객체로 변환하는 전처리를 수행합니다.

In [19]:
import datasets

system_prompt = """You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query."""
user_prompt = """Given the <USER_QUERY>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.

<USER_QUERY>
{question}
</USER_QUERY>"""

def convert_to_conversation(examples):
    train_data = []
    for i in range(len(examples)):
        messages = [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt.format(question=examples["question"][i])
            },
            {
                "role": "assistant",
                "content": examples['query'][i]
            }
        ]
        train_data.append(
            {
                "messages": messages,
            }
        )
    return train_data

train_data_list = convert_to_conversation(train_data)
train_dataset = datasets.Dataset.from_list(train_data_list)
print(train_dataset["messages"][0])

[{'content': 'You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query.', 'role': 'system'}, {'content': "Given the <USER_QUERY>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.\n\n<USER_QUERY>\nHow many heads of the departments are older than 56 ?\n</USER_QUERY>", 'role': 'user'}, {'content': 'SELECT count(*) FROM head WHERE age  >  56', 'role': 'assistant'}]


보통 모델을 학습할 때 사용하는 labels는 inputs와 동일합니다.

(모델은 내부적으로 labels를 한 칸 오른쪽으로 shift한 후 Cross-Entropy loss를 계산합니다. 이는 다음 토큰 예측(next token prediction)과 같은 auto-regressive 분류 작업에서 표준 방식입니다.)

따라서 전처리된 샘플은 다음과 같은 형태가 됩니다:
```python
{
    "input_ids": instruction + model response(assistant),
    "labels": instruction + model response(assistant)
}  # HF 모델이 shift +1 처리를 내부적으로 수행
```

- instruction : 모델에게 입력으로 넣을 지시사항 (실제 User들이 입력으로 넣을 텍스트)
- model response : instruction에 대한 모델의 응답 (`assistant`와 동일)

하지만 우리가 하려는 작업은 instruction 부분을 -100으로 대체하는 것입니다. 왜냐하면 저희는 모델이 정답(`assistant`) 부분만 학습을 하도록 하고 싶기 때문입니다.

```python
{
    "input_ids": instruction + model response(assistant),
    "labels": [-100]*len(instruction) + model response(assistant)
}
```

이렇게 하면 Cross-Entropy 함수에 instruction 토큰은 무시(ignore) 하라고 알려주는 셈입니다. 이를 통해 assistant 부분만 학습을 하도록 만들 수 있습니다.

예를 들어,

```
<|im_start|>system
You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query.<|im_end|>
<|im_start|>user
Given the <USER_QUERY>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.

<USER_QUERY>
How many heads of the departments are older than 56 ?
</USER_QUERY><|im_end|>
<|im_start|>assistant
SELECT count(*) FROM head WHERE age  >  56<|im_end|>
```

위에서 학습해야 할 부분(정답 라벨)은
```
SELECT count(*) FROM head WHERE age  >  56<|im_end|>
```
이 부분이므로, 나머지 부분들은 -100으로 마스킹 처리를 합니다.

In [21]:
import torch

def convert_train_data(examples, tokenizer):
    messages = examples["messages"]
    label_message = messages[-1]["content"]
    label_input_ids = tokenizer.encode(
        label_message, add_special_tokens=False, return_tensors="pt"
    ).squeeze(0)

    # 🚨 핵심 수정 부분: 스퀴즈(squeeze) 에러를 막아주는 방어막 전개!
    prompt_output = tokenizer.apply_chat_template(
        messages[:2], add_generation_prompt=False, tokenize=True, return_tensors="pt"
    )
    if isinstance(prompt_output, dict) or hasattr(prompt_output, "input_ids"):
        prompt_input_ids = prompt_output["input_ids"].squeeze(0)
    else:
        prompt_input_ids = prompt_output.squeeze(0)

    response_start_template_ids = tokenizer.encode("<|im_start|>assistant", return_tensors="pt")[0]

    input_ids = torch.cat(
        [
            prompt_input_ids,
            response_start_template_ids,
            label_input_ids,
            torch.tensor([tokenizer.eos_token_id]),
        ],
        dim=0,
    )
    attention_mask = torch.ones(len(input_ids), dtype=torch.int64)
    labels = torch.cat(
        [
            torch.tensor(
                [-100] * (len(input_ids) - len(label_input_ids) - 1)
            ),  # prompt + label_start_template
            label_input_ids,  # label
            torch.tensor([tokenizer.eos_token_id]),  # [EOS]
        ],
        dim=0,
    )

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

datasets.disable_progress_bar()
train_dataset = train_dataset.map(
    lambda x: convert_train_data(x, tokenizer),
    batched=False,
    num_proc=1,
    remove_columns=train_dataset.column_names  # 원본 컬럼 제거하고 새 컬럼만 남김
)

# 🚨 잘려있던 꼬리 복구: 데이터를 텐서 블록으로 확실하게 변환!
train_dataset.set_format(type="torch")
print("✨ 데이터 변환 완벽 성공! 이제 훈련 가보자고! ✨")

✨ 데이터 변환 완벽 성공! 이제 훈련 가보자고! ✨


1. input_ids – 토크나이즈된 시퀀스 데이터
- 토크나이저(tokenizer)가 텍스트를 **서브워드 단위(subword units)**로 분할하고, 이를 **어휘 사전(vocabulary)**에 정의된 **토큰 인덱스(token index)**로 변환합니다.
- 이 시퀀스가 Transformer 모델의 **임베딩 레이어(embedding layer)**에 입력되어, 각 정수가 고차원 벡터로 임베딩됩니다.

2. attention_mask – 시퀀스 마스킹 텐서
- Transformer는 일반적으로 고정된 최대 길이(max sequence length) 입력을 사용합니다.
- 실제 입력 길이가 짧으면 패딩(padding)을 추가해 맞추는데, 이때 패딩 토큰은 self-attention 연산에서 무시되어야 합니다.
- attention_mask는 같은 길이의 바이너리 벡터로,
  - 실제 토큰 위치 → 1
  - 패딩 토큰 위치 → 0
- 이 마스크는 어텐션 스코어(attention score) 계산 시 소프트맥스 이전에 매우 작은 음수(−∞)를 더해, 패딩 위치의 기여도를 완전히 제거합니다.

3. labels – 학습 대상(target) 시퀀스
- labels는 **모델의 출력 로짓(logits)**과 비교할 타겟 시퀀스입니다.
- 언어 모델 학습 시 auto-regressive(next-token prediction) 방식으로 학습하므로, labels는 보통 input_ids와 동일하지만 손실 계산에서 제외할 토큰 위치를 -100으로 마스킹합니다.
- 이렇게 하면 Cross-Entropy Loss 계산 시 무시된 위치는 loss=0으로 처리됩니다.

In [22]:
print("input_ids : ", train_dataset["input_ids"][0])
print("attention_mask : ", train_dataset["attention_mask"][0])
print("labels : ", train_dataset["labels"][0])

input_ids :  tensor([    1,  9690,   198,  2683,   359,   253,  1694,   288, 15142,  8520,
        31077,    30, 21626,   523,  1998,   346,  2029,   281,  2321,   284,
          346,   523,  5051,   253, 15142,  8520,    30,     2,   198,     1,
         4093,   198, 15423,   260,  2067, 23824,    79, 11745, 33642, 18128,
         5051,   260,  8030, 15142,  3588,   288, 19662,   260,  6253,   940,
           28,  6361,   260,  8520,   506, 17227,    28, 35243,    28,   284,
        17808, 10195,    30,   198,   198,    44, 23824,    79, 11745, 33642,
           46,   198,  2020,   800,  8648,   282,   260, 11609,   359,  3943,
          670,   216,    37,    38,  9148,   198,  9617, 23824,    79, 11745,
        33642,    46,     2,   198,     1,   520,  9531, 23428,   985, 12905,
           25, 18451,  1680, 28531,  1850,   216,  2986,   256,    37,    38,
            2])
attention_mask :  tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1

# 4. base model 성능 측정
- 학습 목표 : spider 벤치마크를 이용해서 base model의 성능을 측정할 수 있다.
- 학습 개념 : metric
- 진행하는 실습 요약
    - 테스트 데이터셋 불러오기
    - 성능 측정

model 학습을 하기 전에 base model의 성능이 얼마인지를 측정합니다.

모델의 성능을 측정하기 위해서는 2가지가 필요합니다.
1. 성능을 측정하기 위한 테스트 데이터셋
2. 성능을 측정하기 위한 지표

spider 데이터셋에 있는 test 데이터셋을 활용하겠습니다.

학습 데이터셋과 동일한 성격을 가진 데이터셋입니다.

따라서, 테스트 데이터셋도 동일하게 필요한 column만 남기도록 하겠습니다.

In [23]:
test_df = text_to_sql["test"].to_pandas()
test_data = test_df[columns]
test_data

,query,question
0,SELECT DISTINCT country FROM singer WHERE age ...,What are all distinct countries where singers ...
1,SELECT DISTINCT country FROM singer WHERE age ...,What are the different countries with singers...
2,SELECT count(*) FROM pets WHERE weight > 10,Find the number of pets whose weight is heavie...
3,SELECT count(*) FROM pets WHERE weight > 10,How many pets have a greater weight than 10?
4,SELECT count(DISTINCT pettype) FROM pets,Find the number of distinct type of pets.
5,SELECT count(DISTINCT pettype) FROM pets,How many different types of pet are there?
6,SELECT count(*) FROM CONTINENTS;,How many continents are there?
7,SELECT count(*) FROM CONTINENTS;,What is the number of continents?
8,SELECT count(*) FROM AIRLINES,How many airlines do we have?
9,SELECT count(*) FROM AIRLINES,What is the total number of airlines?


In [24]:
text_idx = 0

test_input = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt.format(question=test_data["question"][text_idx])},
]
print("정답 :", test_data["query"][text_idx])

정답 : SELECT DISTINCT country FROM singer WHERE age  >  20


이제 base model의 성능을 측정하기 위해 측정 metric을 설정합니다. 저희가 사용할 metric은 `execution accuracy`입니다.

text to sql task의 경우, 자연어를 입력으로 받아 model이 sql문을 출력하는 것이므로, 모델의 response 결과물이 실제 sql문이 잘 동작하는지를 기준으로 평가를 진행할 수 있습니다. 이러한 평가 방식은 sql문은 다르더라도 의도대로 실행이 되었다면 정답이 될 수 있으므로 정확히 SQL문이 일치하는지 평가하는 exact match 보다 유연한 방식입니다.

이러한 metric은 실제로 database에 들어있는 데이터를 sql문을 실행했을 때, 의도한 sql문이 동작하는지를 평가하는 방식입니다. 아래 예시 데이터로 진행해보겠습니다.

다음은 평가 프로세스 상세 흐름입니다.

**Step 1: 파일 준비 및 읽기**

평가를 시작하기 위해 두 개의 텍스트 파일을 준비합니다.

**1-1. 정답 파일(Gold) 읽기**
- 시험지의 정답지와 같은 역할을 합니다
- 각 줄에는 "정답 SQL 쿼리"와 "어떤 데이터베이스를 사용할지"가 탭으로 구분되어 있습니다
- 빈 줄이 나오면 하나의 대화 세션이 끝났다는 의미입니다
- 결과: 여러 세션의 정답들이 리스트로 정리됩니다

**1-2. 예측 파일(Predict) 읽기**
- 학생이 제출한 답안지와 같은 역할입니다
- 각 줄에는 모델이 생성한 SQL 쿼리가 들어있습니다
- 마찬가지로 빈 줄로 세션을 구분합니다
- 결과: 모델의 예측들이 리스트로 정리됩니다

**1-3. 개수 확인**
- 정답과 예측의 개수가 정확히 일치하는지 확인합니다
- 만약 개수가 다르면 평가를 진행할 수 없으므로 오류가 발생합니다

**Step 2: 평가 준비 작업**

**2-1. 평가 도구 준비**
- Evaluator라는 채점 도구를 준비합니다
- 이 도구가 SQL을 분석하고 점수를 계산합니다

**2-2. 점수판 초기화**
- 난이도별 점수판을 만듭니다 (쉬움, 보통, 전체)
- 각 점수판에는 다음 정보를 담을 공간을 마련합니다:
  - 문제 개수
  - 실행 정확도 (실제로 돌려봤을 때 맞는지)
  - 구조 정확도 (SQL 문법 구조가 맞는지)
  - 부분 점수 (SELECT, WHERE 등 각 부분별 점수)

**Step 3: 본격적인 채점 시작**

모든 SQL 쿼리 쌍(정답과 예측)에 대해 하나씩 채점을 진행합니다.

**Step 3-1: SQL 이해하기 (파싱)**

**정답 SQL 분석**
- 데이터베이스의 구조 정보(스키마)를 먼저 불러옵니다
  - 어떤 테이블이 있는지
  - 각 테이블에 어떤 컬럼이 있는지
  - 테이블 간 관계(외래키)가 어떻게 되는지

- 정답 SQL 문자열을 컴퓨터가 이해할 수 있는 구조화된 형태로 변환합니다
  - 예: "SELECT name FROM students WHERE age > 20"
  - 이것을 → SELECT 부분은 무엇, FROM 부분은 무엇, WHERE 조건은 무엇... 이렇게 분해합니다

**예측 SQL 분석**
- 모델이 생성한 SQL도 같은 방식으로 분석합니다
- 만약 SQL 문법이 잘못되어 분석이 불가능하면, 빈 SQL로 간주합니다 (0점 처리)

**Step 3-2: 난이도 판정**

정답 SQL을 보고 이 문제가 얼마나 어려운지 자동으로 판단합니다.

**난이도 계산 기준**
- **기본 요소 카운트**: WHERE, GROUP BY, ORDER BY, LIMIT, JOIN, OR, LIKE 같은 것들이 몇 개 있는지
- **복잡도 카운트**: 서브쿼리(중첩 쿼리)가 있는지, UNION/INTERSECT/EXCEPT가 있는지
- **추가 요소 카운트**: 집계함수를 여러 번 쓰는지, SELECT에 여러 컬럼이 있는지 등

**난이도 등급**
- **쉬움(easy)**: 기본적인 SELECT와 간단한 조건 하나 정도
- **보통(medium)**: 조건이 2~3개 있거나 JOIN이 하나 정도 있는 수준

**Step 3-3: 실행 결과로 채점 (Execution 평가)**

실제로 SQL을 데이터베이스에서 실행해서 결과가 같은지 확인합니다.

**실행 평가 과정**
1. 정답 SQL을 실행해서 결과 테이블 A를 얻습니다
2. 예측 SQL을 실행해서 결과 테이블 B를 얻습니다
3. 여러 개의 테스트 데이터베이스에서 반복 실행합니다
   - 왜냐하면 우연히 하나의 데이터베이스에서만 맞을 수도 있으니까요
4. 모든 테스트에서 A와 B가 완전히 동일하면 정답
5. 하나라도 다르면 오답

**점수 기록**
- 맞으면 해당 난이도, 해당 턴, 전체 점수에 1점 추가
- 틀리면 0점

**Step 3-4: 구조로 채점 (Exact Match 평가)**

SQL의 구조가 정답과 정확히 같은지 세밀하게 비교합니다.

**전처리 작업**
- 값(Value)을 무시하도록 설정합니다
  - 예: "age > 20"과 "age > 30"을 같은 구조로 봅니다 (숫자는 달라도 됨)
- 외래키 관계를 고려해서 컬럼을 정리합니다
  - 예: `students.dept_id`와 `departments.id`가 같은 것을 의미할 수 있음

**각 부분별 상세 채점**

1. **SELECT 절 채점**
   - 선택한 컬럼들이 정답과 같은지 확인
   - 집계함수(COUNT, SUM, AVG 등) 사용이 맞는지 확인
   - 순서는 상관없이 내용만 비교

2. **WHERE 절 채점**
   - 조건들이 정답과 같은지 확인
   - 비교 연산자(=, >, <, LIKE 등)가 맞는지 확인
   - 여러 조건이 있으면 하나씩 체크

3. **GROUP BY 절 채점**
   - 그룹핑하는 컬럼이 맞는지 확인
   - 테이블명은 무시하고 컬럼명만 비교

4. **ORDER BY 절 채점**
   - 정렬 기준 컬럼이 맞는지 확인
   - 오름차순/내림차순이 맞는지 확인
   - LIMIT과 함께 사용되었는지도 체크

5. **JOIN 채점**
   - 어떤 테이블들을 연결했는지 확인
   - 연결 조건이 맞는지 확인

6. **AND/OR 채점**
   - 조건들을 연결하는 방식이 맞는지 확인

7. **서브쿼리 채점**
   - UNION, INTERSECT, EXCEPT 같은 복잡한 쿼리가 맞는지 확인
   - 서브쿼리 내부도 재귀적으로 같은 방식으로 채점

**최종 판정**
- 모든 부분이 100% 일치하면 Exact Match 성공 (1점)
- 하나라도 다르면 실패 (0점)
- 각 부분별 점수도 따로 기록 (부분 점수)

**Step 4: 결과 정리 및 출력**

모든 쿼리에 대한 채점이 끝나면 최종 점수를 계산합니다.

**4-1. 평균 계산**
- 각 난이도별로 정확도를 계산합니다
  - 예: 쉬운 문제 100개 중 85개 맞음 → 85% 정확도
- 실행 정확도와 구조 정확도를 각각 계산합니다
- 부분 점수도 계산합니다
  - Accuracy(정확도): 예측한 것 중 맞은 비율
  - Recall(재현율): 정답에 있는 것 중 찾은 비율
  - F1 점수: Accuracy와 Recall의 조화평균

**4-2. 결과 출력**
화면에 보기 좋게 표 형태로 결과를 출력합니다:


In [25]:
# Powershell
!powershell "cd test-suite-sql-eval-master ; python evaluation.py --gold gold.txt --pred base_model_predict.txt --db database/ --etype exec --plug_value"
# git bash
# !cd test-suite-sql-eval-master && python evaluation.py --gold gold.txt --pred base_model_predict.txt --db database/ --etype exec --plug_value

/bin/bash: line 1: powershell: command not found


# 5. Text-to-SQL task 모델 QLoRA fine-tuning
- 학습 목표 : text-to-sql task에 대한 QLoRA fine-tuning을 진행할 수 있다.
- 학습 개념 : QLoRA, Trainer
- 진행하는 실습 요약
    - QLoRA fine-tuning
    - 성능 측정

QLoRA 설정과 Trainer 설정을 하고 모델 학습을 진행합니다.

QLoRA는 Quantization 모델에 LoRA 학습을 진행하는 것이므로 LoRA 설정을 그대로 진행하면 됩니다.

LoRA config 설정을 합니다.

참고 문서에서는 특히 r, lora_alpha, lora_dropout, target_modules 항목을 중심으로, LoRA 적용 범위와 학습 효율·메모리 사용량에 미치는 영향을 확인해주세요.

In [26]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]  # safe default for many LLMs
)

### SFTConfig 주요 설정 설명
각 설정들은 기본적인 AI에 대한 지식이 필요한 부분들이 많습니다. 따라서, 각 설정들은 하나하나 블로그 글을 참고해가며 깊이 이해해주세요.

1. output_dir
- 설명: 학습이 끝난 모델 가중치, 체크포인트, 로그를 저장할 디렉터리 (또는 Hugging Face Hub 저장소 이름)
- 중요성: 실험 결과를 재현하거나 이어서 학습하려면 필수로 설정해야 함

2. max_seq_length
- 설명: 한 학습 샘플의 최대 토큰 길이
- 영향:
	1. 길이를 크게 하면 더 긴 문장을 학습할 수 있으나 GPU 메모리 사용량 증가
	2. 너무 작으면 긴 문장이 잘려서 학습 품질 저하

3. packing
	- 설명: 여러 짧은 샘플을 하나의 시퀀스로 이어 붙여(max_seq_length까지) 학습
	- 장점: GPU 메모리 효율 ↑, 학습 속도 ↑ (패딩 낭비 감소)
	- 주의점: 시퀀스 사이를 구분하는 special token(EOS 등)을 넣어야 데이터 섞임 방지
  - PACKING 예시

    데이터셋 샘플이 각각 독립된 시퀀스로 학습되고 학습할 토큰들이 짧으면 패딩이 많아집니다.

    ```
    샘플 1: [USER] 오늘 날씨 어때? [EOS] [PAD] [PAD] [PAD] ...
    샘플 2: [USER] 내일 비 와? [EOS] [PAD] [PAD] [PAD] ...
    샘플 3: [USER] 주말 일정 보여줘 [EOS] [PAD] [PAD] ...
    ```
    GPU는 [PAD]도 연산해야 하므로 메모리 낭비 & 속도 저하가 발생합니다.

    짧은 샘플들을 이어붙이고, 샘플 사이에 EOS 같은 구분 토큰을 추가하여 최대 길이(max_seq_length)에 맞게 하나의 긴 시퀀스로 묶습니다.

    Packed 시퀀스:
    ```
    [USER] 오늘 날씨 어때? [EOS]
    [USER] 내일 비 와? [EOS]
    [USER] 주말 일정 보여줘 [EOS]
    ```
    - 패딩이 거의 없고 GPU 연산 효율 ↑
    - 한 시퀀스 안에서 여러 예시를 한 번에 학습 → 학습 속도 ↑
    - `Packing=False` : `[샘플1][PAD PAD PAD] [샘플2][PAD PAD PAD]`
    - `Packing=True` : `[샘플1][EOS][샘플2][EOS][샘플3][EOS]	`

4. num_train_epochs
- 설명: 전체 데이터셋을 몇 번 반복해서 학습할지 결정
- 팁: LoRA/QLoRA는 적은 Epoch(2~3)으로도 잘 수렴하는 경우가 많음

5. per_device_train_batch_size
- 설명: GPU 한 장(또는 장치 하나)에서 한 번에 처리할 샘플 개수
- 조절 포인트: GPU 메모리에 맞춰 조절. 작게 하면 gradient_accumulation_steps로 보완 가능

6. gradient_accumulation_steps
- 설명: 여러 스텝의 그래디언트를 누적해 한 번만 역전파/가중치 업데이트
- 장점: 작은 배치로도 큰 effective batch size 구현 가능
- 예시: batch_size=1, grad_accum=4 → 실제 batch size = 4와 비슷한 효과
- `gradient_accumulation_steps`은 [링크]()를 참고해주세요.

7. gradient_checkpointing
- 설명: 순전파(forward) 중 일부 중간 값을 저장하지 않고 필요할 때 다시 계산
- 장점: GPU 메모리 절약 (특히 대형 모델 학습 시 필수)
- 단점: 연산량 증가 → 학습 속도 소폭 느려짐

8. optim
- 설명: 사용할 옵티마이저 선택 (adamw_torch_fused는 PyTorch의 fused AdamW)
- 장점: 일반 AdamW보다 빠르고 메모리 효율 좋음 (지원 GPU 필요)

9. logging_steps
- 설명: 몇 step마다 학습 로그를 기록할지
- 팁: 너무 작으면 로그 과다, 너무 크면 학습 추적 힘듦 → 10~50 step 정도 권장

10. save_strategy
- 설명: 모델 체크포인트 저장 시점 (epoch, steps, no)
- 예시: "epoch" → 각 epoch가 끝날 때마다 저장

11. learning_rate
- 설명: 모델 학습률
- 팁: QLoRA 논문에서 2e-4가 안정적이라고 권장
- 주의: 지나치게 높으면 loss 폭주, 낮으면 학습 느림

12. fp16 / bf16
- 설명: 학습 시 연산 정밀도 선택
- fp16: 대부분 GPU 지원, 메모리 절약, 속도 빠름
- bf16: 최신 GPU(Ampere 이상)에서 권장, 안정성 더 좋음
- 팁: Colab A100/T4 → fp16 / A100(Ampere) 이상 → bf16 추천

13. max_grad_norm
- 설명: Gradient clipping 값 (너무 큰 gradient를 잘라 학습 안정화)
- QLoRA 권장값: 0.3

14. warmup_ratio
- 설명: 학습 초기에 learning rate를 천천히 올리는 비율
- 장점: 학습 안정화, 초기 손실 폭주 방지
- QLoRA 논문 값: 0.03 (~3% 단계는 warmup)

15. lr_scheduler_type
- 설명: 학습률 스케줄링 방식
- constant: 학습 내내 일정한 학습률
- (다른 옵션: linear, cosine 등)

16. push_to_hub
- 설명: 학습이 끝난 모델을 Hugging Face Hub에 자동 업로드할지 여부
- 협업/공유: 팀원이나 공개 프로젝트에 유용

17. report_to
- 설명: 학습 메트릭을 어디로 보낼지 (예: "tensorboard", "wandb")
- 장점: 학습 과정 시각화 가능

학습 설정을 진행합니다.

In [28]:
# AI 훈련 도구 다시 강제 주입!
!pip install -U trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 12.3 MB/s eta 0:00:00


In [42]:
from trl import SFTConfig, SFTTrainer

output_dir = "outputs"
num_train_epochs=1
per_device_train_batch_size = 4 # GPU가 부족하다면 해당 값을 줄여주세요.
gradient_accumulation_steps = 4
warmup_ratio = 0.03
max_steps = 100 # 학습 시간이 오래 걸린다면 해당 값을 줄여주세요.
learning_rate = 2e-5
logging_steps = 1
weight_decay = 0.01
max_grad_norm=1.0
lr_scheduler_type = "linear"
report_to = "none" # Use this for WandB etc
bf16=False
gradient_checkpointing=False
optim="adamw_torch"

train_cfg = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    weight_decay=weight_decay,
    logging_steps=logging_steps,
    save_steps=max_steps,
    max_steps=max_steps,
    fp16=False,
    bf16=False,
    report_to=report_to,
    gradient_checkpointing=gradient_checkpointing,
    optim=optim,
)

warmup_ratio is deprecated and will be removed in v5.2. Use `warmup_steps` instead.


학습을 진행합니다.

In [43]:
trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset = train_dataset.select(range(max_steps * per_device_train_batch_size)), # max_steps * batch_size 만큼만 학습합니다.
    peft_config=peft_config,
    args = train_cfg,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:285: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [44]:
# 반드시 fp16은 True, bf16은 False여야 합니다!
train_cfg.fp16 = True
train_cfg.bf16 = False

In [45]:
# === 학습 효율성 측정 코드 ===
import time
import torch

def measure_training_efficiency(trainer, max_steps):
    """LoRA vs QLoRA 효율성 비교를 위한 측정 함수"""

    # 학습 시간 측정
    start_time = time.time()
    trainer_stats = trainer.train()
    print(trainer_stats)
    training_time = time.time() - start_time

    return {
        "학습 시간 (초)": round(training_time, 2),
        "초당 스텝 수": round(max_steps / training_time, 3),
    }

# 결과 출력
results = measure_training_efficiency(trainer, max_steps)
print("=== 학습 효율성 측정 결과 ===")
for k, v in results.items():
    print(f"{k}: {v}")

Step,Training Loss
1,2.126898
2,2.374540
3,2.703796
4,1.980945
5,1.894700
6,2.146062
7,2.169174
8,2.677753
9,1.959607
10,2.070188


TrainOutput(global_step=100, training_loss=1.5115701740980148, metrics={'train_runtime': 1024.3027, 'train_samples_per_second': 1.562, 'train_steps_per_second': 0.098, 'total_flos': 2415798587572224.0, 'train_loss': 1.5115701740980148})
=== 학습 효율성 측정 결과 ===
학습 시간 (초): 1025.09
초당 스텝 수: 0.098


스켈레톤 1에서는 약 2분 내에 학습이 끝났지만, 스켈레톤 2에서는 약 6분 이상의 학습 시간이 걸립니다.

QLoRA는 더 큰 모델을 학습할 수 있지만, 양자화와 역양자화를 거치기 때문에 학습시간이 엄청나게 비효율적으로 증가하게 됩니다.

또한, 메모리를 체크해보면면 학습 메모리는 스켈레톤 2와 크게 달라지지 않았지만, 큰 모델을 학습시킬 수 있다는 것을 확인할 수 있습니다.

In [46]:
!nvidia-smi

Tue Feb 24 02:43:24 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   62C    P0             30W /   70W |    5933MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

LoRA를 저장하는 방식은 두가지가 있습니다.
1. base model에 LoRA weight를 merge해서 저장하는 방식
2. LoRA weight만 따로 저장하는 방식

두 방식 모두 가능하지만, 여기서는 2번 방식으로 진행하겠습니다. 많은 학습을 돌린다면 1번보다는 2번이 더욱 비용 효율적입니다.

In [47]:
# ============================================
# Save LoRA adapter (small)
# ============================================
merge_and_save = False
if merge_and_save:
    trainer.model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"LoRA adapter saved to: {output_dir}")

# ============================================
# (Optional) Merge LoRA into base weights and save a full model
# WARNING: creates a large model; only do this if you need standalone weights
# ============================================
if merge_and_save:
    from peft import AutoPeftModelForCausalLM
    merged_model = AutoPeftModelForCausalLM.from_pretrained(
        output_dir,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
        device_map="auto",
    )
    merged_model = merged_model.merge_and_unload()
    merged_dir = output_dir + "-merged"
    merged_model.save_pretrained(merged_dir, safe_serialization=True)
    tokenizer.save_pretrained(merged_dir)
    print(f"Merged full model saved to: {merged_dir}")


모델 결과물이 잘 나오는지 테스트를 진행합니다. 우선 학습 데이터로 학습이 잘 되었는지 확인만 해보고 실제 모델의 테스트는 평가를 통해 진행합니다.

In [48]:
import time

text_idx = len(train_df) - 1

test_input = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt.format(question=train_df["question"][text_idx])},
]
print("정답 :", train_df["query"][text_idx])

inputs = tokenizer.apply_chat_template(test_input, add_generation_prompt=True, return_dict=True, return_tensors="pt")
input_prompt = tokenizer.apply_chat_template(test_input, add_generation_prompt=True, tokenize=False)

# 추론 속도 측정 시작
torch.cuda.synchronize()  # GPU 작업 완료 대기
start_time = time.perf_counter()

with torch.no_grad():
    output_ids = model.generate(
        **inputs.to("cuda"),
        max_new_tokens=128,
        stop_strings=["<|endofturn|>", "<|stop|>"],
        tokenizer=tokenizer
    )

# 추론 속도 측정 종료
torch.cuda.synchronize()  # GPU 작업 완료 대기
end_time = time.perf_counter()

# 추론 시간 계산
inference_time = end_time - start_time
input_token_count = inputs["input_ids"].shape[1]
output_token_count = output_ids.shape[1]
generated_token_count = output_token_count - input_token_count
tokens_per_second = generated_token_count / inference_time

print(tokenizer.batch_decode(output_ids)[0][len(input_prompt) - 1:])

# 추론 속도 출력
print("\n" + "="*50)
print(f"📊 추론 속도 측정 결과")
print("="*50)
print(f"⏱️  총 추론 시간: {inference_time:.3f}초")
print(f"📥 입력 토큰 수: {input_token_count}개")
print(f"📤 생성 토큰 수: {generated_token_count}개")
print(f"🚀 토큰 생성 속도: {tokens_per_second:.2f} tokens/sec")
print("="*50)

정답 : SELECT T2.company_name FROM movie AS T1 JOIN culture_company AS T2 ON T1.movie_id  =  T2.movie_id WHERE T1.year  =  1999

SELECT company_name FROM company_movies WHERE year = 1999 AND movie_name = 'The Matrix'<|im_end|>

📊 추론 속도 측정 결과
⏱️  총 추론 시간: 7.304초
📥 입력 토큰 수: 105개
📤 생성 토큰 수: 27개
🚀 토큰 생성 속도: 3.70 tokens/sec


In [49]:
!zip -r /content/trained_model.zip /content/outputs/

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/checkpoint-100/ (stored 0%)
  adding: content/outputs/checkpoint-100/rng_state.pth (deflated 26%)
  adding: content/outputs/checkpoint-100/tokenizer_config.json (deflated 49%)
  adding: content/outputs/checkpoint-100/optimizer.pt (deflated 21%)
  adding: content/outputs/checkpoint-100/training_args.bin (deflated 53%)
  adding: content/outputs/checkpoint-100/chat_template.jinja (deflated 42%)
  adding: content/outputs/checkpoint-100/tokenizer.json (deflated 82%)
  adding: content/outputs/checkpoint-100/scheduler.pt (deflated 62%)
  adding: content/outputs/checkpoint-100/adapter_config.json (deflated 58%)
  adding: content/outputs/checkpoint-100/trainer_state.json (deflated 81%)
  adding: content/outputs/checkpoint-100/README.md (deflated 65%)
  adding: content/outputs/checkpoint-100/adapter_model.safetensors (deflated 23%)
  adding: content/outputs/README.md (deflated 43%)


이제 모델 성능을 평가해봅시다.

### step 1.
모델 성능을 평가하기 이전에 jupyter notebook은 커널을 계속 실행하고 있기 때문에 jupyter notebook을 `restart`를 해야만 jupyter notebook에서 사용하는 GPU가 비워집니다.

꼭 jupyter notebook을 비우고 `python inference.py` 명령어를 실행해주세요. `outputs/` 디렉토리가 있다면 LoRA 모델을 먼저 불러오게끔 설정이 되어 있습니다.

Powershell 혹은 bash에서 스크립트를 실행하게 되는데 가상환경을 꼭 실행하고 진행해주세요.
```bash
source .venv/Scripts/activate
```

### step 2.
아래 명령어로 inference.py를 돌리실 수 있습니다.
```bash
python inference.py
```

### step 3.

inference를 돌린 이후에는
```bash
cd test-suite-sql-eval-master
python evaluation.py --gold gold.txt --pred predict.txt --db database/ --etype exec --plug_value
```
명령어로 평가를 직접 진행해보시길 바랍니다.